# 08. 배달 분석 + 성동땡겨요 정책

## 목적
- 배달 활용도별 폐업률 분석
- 땡겨요 입점 타겟 식별
- 정책 효과 시뮬레이션

In [1]:
import pandas as pd
import numpy as np

labeled_data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/1_시간라벨링/labeled_data.csv"
df = pd.read_csv(labeled_data_path)

cluster_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석/클러스터링_결과_완전판.csv"
cluster_df = pd.read_csv(cluster_path)

df = df.merge(cluster_df[['가맹점구분번호', 'cluster']], on='가맹점구분번호', how='left')
df['위험그룹'] = df['cluster'].apply(lambda x: '고위험' if x in [0, 2] else '정상' if x == 1 else '알수없음')

print(f"데이터: {len(df):,}개")

데이터: 89,412개


## 배달 활용도 분류

In [2]:
if '배달매출금액 비율' in df.columns:
    배달비율 = df['배달매출금액 비율'].fillna(0)
    
    def classify_delivery_usage(ratio):
        if ratio == 0:
            return '미활용 (0%)'
        elif ratio <= 20:
            return '저활용 (1-20%)'
        elif ratio <= 50:
            return '중활용 (21-50%)'
        else:
            return '고활용 (50%+)'
    
    df['배달활용도그룹'] = 배달비율.apply(classify_delivery_usage)
    print("\n[배달 활용도 그룹별 분포]")
    print(df['배달활용도그룹'].value_counts().sort_index())


[배달 활용도 그룹별 분포]
배달활용도그룹
고활용 (50%+)       6995
미활용 (0%)        62892
저활용 (1-20%)     13186
중활용 (21-50%)     6339
Name: count, dtype: int64


## 배달 활용도별 폐업률

In [3]:
df['is_closed'] = df['폐업일'].notna().astype(int)

group_closure = df.groupby('배달활용도그룹').agg({
    'is_closed': ['sum', 'mean', 'count']
}).round(4)

group_closure.columns = ['폐업수', '폐업률', '가맹점수']
group_closure = group_closure.sort_values('폐업률', ascending=False)

print("\n[배달 활용도별 폐업률]")
print(group_closure)


[배달 활용도별 폐업률]
               폐업수     폐업률   가맹점수
배달활용도그룹                          
고활용 (50%+)     794  0.1135   6995
저활용 (1-20%)    951  0.0721  13186
중활용 (21-50%)   360  0.0568   6339
미활용 (0%)      3378  0.0537  62892


## 땡겨요 타겟 식별

In [4]:
target_df = df[(df['배달활용도그룹'] == '미활용 (0%)') & (df['위험그룹'] == '고위험')].copy()

print(f"\n땡겨요 입점 타겟: {len(target_df):,}개")
print(f"전체 대비 비율: {len(target_df) / len(df) * 100:.2f}%")


땡겨요 입점 타겟: 62,346개
전체 대비 비율: 69.73%


### 정책적 의미
- 배달 미활용 + 고위험: 땡겨요 입점으로 개선 가능
- 수수료 2% 절감 + 15% 할인 효과 기대